In [1]:
import pandas as pd
import psycopg2
import warnings

In [4]:
conn_string = "host='localhost' dbname='video_games' user='postgres' password='56067031'"
conn = psycopg2.connect(conn_string)
warnings.filterwarnings('ignore', category=Warning)

### PRE DECLARING SCHEMA AND TABLE NAMES SO THAT IT IS EASIER TO QUERY THEM

In [22]:
schema_name = 'video_games'
game = 'game'
game_platform = 'game_platform'
game_publisher = 'game_publisher'
genre = 'genre'
publisher = 'publisher'
region = 'region'
region_sales = 'region_sales'
platform = 'platform'

#### 1. GAME TABLE

In [17]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{game}", conn).head()

,id,genre_id,game_name
0,1,11,98 Koshien
1,2,8,.hack//G.U. Vol.1//Rebirth
2,3,8,.hack//G.U. Vol.2//Reminisce
3,4,8,.hack//G.U. Vol.2//Reminisce (jp sales)
4,5,8,.hack//G.U. Vol.3//Redemption


#### 2. GAME PLATFORM DETAILS TABLE

In [16]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{game_platform}", conn).head()

,id,game_publisher_id,platform_id,release_year
0,1,8564,4,2007
1,2,9852,4,2007
2,3,11063,7,2006
3,4,9065,15,2011
4,5,9544,15,2011


#### 3. GAME PUBLISHER DETAILS TABLE

In [15]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{game_publisher}", conn).head()

,id,game_id,publisher_id
0,1,10866,369
1,2,9244,369
2,3,5464,369
3,4,10868,369
4,5,7282,369


#### 4. GENRE DETAILS TABLE

In [9]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{genre}", conn).head()

,id,genre_name
0,1,Action
1,2,Adventure
2,3,Fighting
3,4,Misc
4,5,Platform


#### 5. PUBLISHER DETAILS TABLE

In [14]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{publisher}", conn).head()

,id,publisher_name
0,1,10TACLE Studios
1,2,1C Company
2,3,20th Century Fox Video Games
3,4,2D Boy
4,5,3DO


#### 6. REGION DETAILS TABLE

In [11]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{region}", conn).head()

,id,region_name
0,1,North America
1,2,Europe
2,3,Japan
3,4,Other


#### 7. REGION SALES DETAILS TABLE

In [21]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{region_sales}", conn).head()

,region_id,game_platform_id,num_sales
0,1,50,3.50
1,1,51,1.43
2,1,52,0.51
3,1,53,0.27
4,1,54,0.48


#### 8. PLATFORM DETAILS TABLE

In [20]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{platform}", conn).head()

,id,platform_name
0,1,Wii
1,2,NES
2,3,GB
3,4,DS
4,5,X360


## SCHEMA OF THE TABLES
![SCHEMA](SCHEMA.png)

## DATA ANLYSIS QUESTIONS
### EASY
#### 1. Who is the top game publisher based on the number of games published?

In [123]:
pd.read_sql_query(f"""
SELECT gp.publisher_id, pub.publisher_name, COUNT(DISTINCT gp.game_id)
FROM {schema_name}.game_publisher as gp
INNER JOIN {schema_name}.publisher as pub
ON gp.publisher_id = pub.id
GROUP BY publisher_id, pub.publisher_name
ORDER BY COUNT(DISTINCT game_id) DESC
LIMIT 1;
""", conn)

,publisher_id,publisher_name,count
0,352,Namco Bandai Games,774


#### 2. Which platform has the highest total sales across all regions?

In [72]:
pd.read_sql_query(f"""
SELECT gp.platform_id as "Platform Id", plf.platform_name as "Platform Name", SUM(rs.num_sales) as "Total Sales"
FROM {schema_name}.game_platform as gp
INNER JOIN {schema_name}.platform as plf
ON gp.platform_id = plf.id
INNER JOIN {schema_name}.region_sales as rs
ON gp.id = rs.game_platform_id
GROUP BY platform_id, plf.platform_name
ORDER BY SUM(rs.num_sales) DESC
LIMIT 1;
""", conn)

,Platform Id,Platform Name,Total Sales
0,7,PS2,1233.56


#### 3. How many games are categorized under each genre?

In [137]:
pd.read_sql_query(f"""
SELECT gn.id as "Genre ID", gn.genre_name as "Genre Name", COUNT(gm.game_name) as "Number of Games"
FROM {schema_name}.genre as gn
INNER JOIN {schema_name}.game as gm
ON gn.id = gm.genre_id
GROUP BY gn.id, gn.genre_name
ORDER BY COUNT(gm.game_name) DESC;
""", conn)

,Genre ID,Genre Name,Number of Games
0,1,Action,1900
1,11,Sports,1366
2,4,Misc,1314
3,8,Role-Playing,1206
4,2,Adventure,1038
5,9,Shooter,806
6,7,Racing,761
7,10,Simulation,714
8,3,Fighting,605
9,5,Platform,580


#### 4. Which region has the highest total number of sales?

In [71]:
pd.read_sql_query(f"""
SELECT rg.id as "Region ID", rg.region_name as "Region Name", SUM(rs.num_sales) as "Number of Sales"
FROM {schema_name}.region as rg
INNER JOIN {schema_name}.region_sales as rs
ON rg.id = rs.region_id
GROUP BY rg.id, rg.region_name
ORDER BY SUM(rs.num_sales) DESC
LIMIT 1;
""", conn)

,Region ID,Region Name,Number of Sales
0,1,North America,4335.07


#### 5. Which game has the highest sales in North America?

In [75]:
# We know from the region table that North America has id = 1
pd.read_sql_query(f"""
SELECT gm.id, gm.game_name, gpf.game_publisher_id, rs.region_id, SUM(rs.num_sales)
FROM {schema_name}.game as gm
INNER JOIN {schema_name}.game_publisher as gp
ON gm.id = gp.game_id
INNER JOIN {schema_name}.game_platform as gpf
ON gp.id = gpf.game_publisher_id
INNER JOIN {schema_name}.region_sales as rs
ON gpf.id = rs.game_platform_id
WHERE rs.region_id = 1
GROUP BY gm.id,gm.game_name, gpf.game_publisher_id, rs.region_id
ORDER BY SUM(rs.num_sales) DESC
LIMIT 1;
""", conn)

,id,game_name,game_publisher_id,region_id,sum
0,10866,Wii Sports,1,1,41.49


#### 6. How many games were released in each year?

In [105]:
pd.read_sql_query(f"""
SELECT gp.release_year as "Release Year", COUNT(DISTINCT gp.game_publisher_id) as "Number of Games Released"
FROM {schema_name}.game_platform as gp
GROUP BY gp.release_year
ORDER BY COUNT(*) DESC;
""", conn)

,Release Year,Number of Games Released
0,2009,1013
1,2008,1051
2,2010,914
3,2007,900
4,2011,787
5,2006,777
6,2005,678
7,2002,645
8,2003,560
9,2004,566


#### 7. Which is the popularity of different game genres based on number of sales ?

In [156]:
pd.read_sql_query(f"""
SELECT gn.genre_name as "Genre Name", SUM(rs.num_sales) as "Total Number of Sales"
FROM {schema_name}.genre as gn
INNER JOIN {schema_name}.game as gm
ON gn.id = gm.genre_id
INNER JOIN {schema_name}.game_publisher as gp
ON gm.id = gp.game_id
INNER JOIN {schema_name}.game_platform as gpf
ON gp.id = gpf.game_publisher_id
INNER JOIN {schema_name}.region_sales as rs
ON gpf.id = rs.game_platform_id
GROUP BY gn.genre_name
ORDER BY SUM(rs.num_sales) DESC;
""", conn)

,Genre Name,Total Number of Sales
0,Action,1722.05
1,Sports,1308.83
2,Shooter,1025.75
3,Role-Playing,923.75
4,Platform,828.82
5,Misc,796.83
6,Racing,729.79
7,Fighting,444.08
8,Simulation,389.88
9,Puzzle,241.69


#### 8. Which publisher has the highest total sales in each region?

In [165]:
pd.read_sql_query(f"""
SELECT game_publisher_id
FROM {schema_name}.game_platform
WHERE id IN (
SELECT game_platform_id
FROM {schema_name}.region_sales
WHERE (region_id,num_sales) IN (
SELECT rs.region_id,MAX(num_sales)
FROM {schema_name}.region_sales as rs
GROUP BY rs.region_id
ORDER BY rs.region_id ASC))
""", conn)

,game_publisher_id
0,18
1,13
2,1


In [164]:
pd.read_sql_query(f"""
SELECT *
FROM {schema_name}.region_sales
WHERE game_platform_id = 9704
""", conn)

,region_id,game_platform_id,num_sales
0,1,9704,11.27
1,2,9704,8.89
2,4,9704,1.00
3,3,9704,3.50


In [166]:
-- Active: 1686816882614@@127.0.0.1@5432@video_games@video_games
SELECT game_platform_id
FROM region_sales
WHERE (region_id,num_sales) IN (
SELECT rs.region_id,MAX(num_sales)
FROM region_sales as rs
GROUP BY rs.region_id
ORDER BY rs.region_id ASC);

SyntaxError: illegal target for annotation (2623883101.py, line 1)

<img src="SCHEMA.png"  width="1200" height="500">

In [ ]:
-- Active: 1686816882614@@127.0.0.1@5432@video_games@video_games
SELECT *
FROM platform;